In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score

# Pregunta #6

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder

# Cargar los datos
df = pd.read_csv("train.csv")  # Cambia el nombre si es diferente

# Variable objetivo
y = df["SalePrice"]

# 🔸 Eliminar columnas no numéricas o convertirlas a números
df_numeric = df.select_dtypes(include=[np.number])

# 🔸 Opcional: Eliminar la columna Id si no aporta al modelo
if "Id" in df_numeric.columns:
    df_numeric = df_numeric.drop("Id", axis=1)

# 🔸 Separar las características (X)
X = df_numeric.drop("SalePrice", axis=1)

# 🔸 Imputar NaN con la media
imputer = SimpleImputer(strategy="mean")
X_imputed = imputer.fit_transform(X)

# 🔸 Binarizar y: por ejemplo, casas caras vs. baratas
# Aquí hacemos clasificación, no regresión → convertimos SalePrice a 0 o 1
y_binary = (y > y.median()).astype(int)  # 1 = caro, 0 = barato

# 🔸 Separar datos
X_train, X_test, y_train, y_test = train_test_split(X_imputed, y_binary, test_size=0.2, random_state=42)

# 🔸 Entrenar modelo de regresión logística
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# 🔸 Evaluación
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

print(f"Precisión en entrenamiento: {accuracy_score(y_train, y_pred_train):.2f}")
print(f"Precisión en prueba: {accuracy_score(y_test, y_pred_test):.2f}")
print("Matriz de confusión:")
print(confusion_matrix(y_test, y_pred_test))


Precisión en entrenamiento: 0.90
Precisión en prueba: 0.92
Matriz de confusión:
[[147  14]
 [  8 123]]


c:\Users\Usuario\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


# Pregunta 7

In [6]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Definir el modelo de regresión logística
model = LogisticRegression(solver='liblinear')  # 'liblinear' es un buen solver para pequeña/muy grande regularización

# Definir el espacio de parámetros a explorar
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],  # Parámetro de regularización
    'penalty': ['l1', 'l2'],       # Tipo de regularización
    'max_iter': [100, 200, 500]    # Número máximo de iteraciones
}

# Realizar la búsqueda de los mejores parámetros utilizando GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Mostrar los mejores parámetros encontrados
print(f"Mejores parámetros: {grid_search.best_params_}")

# Evaluar el modelo con los mejores parámetros
best_model = grid_search.best_estimator_

# Precisión en entrenamiento
train_accuracy = accuracy_score(y_train, best_model.predict(X_train))
print(f"Precisión en entrenamiento: {train_accuracy:.2f}")

# Precisión en prueba
test_accuracy = accuracy_score(y_test, best_model.predict(X_test))
print(f"Precisión en prueba: {test_accuracy:.2f}")

# Matriz de confusión
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, best_model.predict(X_test))
print("Matriz de confusión:")
print(cm)


c:\Users\Usuario\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\Usuario\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\Usuario\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\Usuario\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\Usuario\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iteration

Mejores parámetros: {'C': 10, 'max_iter': 500, 'penalty': 'l1'}
Precisión en entrenamiento: 0.92
Precisión en prueba: 0.93
Matriz de confusión:
[[150  11]
 [  8 123]]


# Pregunta #8

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import pandas as pd
import numpy as np

# Simulación de datos de ejemplo con NaNs (puedes reemplazar esto con tu propio dataset)
# X = pd.read_csv('tu_archivo.csv')  # Si tienes tu dataset real
np.random.seed(42)
X = pd.DataFrame({
    'feature1': [1, 2, np.nan, 4, 5, 6, 7, 8, 9, 10],
    'feature2': [5, 3, 2, 7, 6, np.nan, 8, 2, 4, 1]
})
y = [0, 1, 0, 1, 0, 1, 0, 1, 0, 1]

def ejecutar_modelo():
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    parametros = {
        'clf__C': [0.01, 0.1, 1, 10],
        'clf__penalty': ['l1'],
        'clf__max_iter': [100, 200, 300, 500]
    }

    pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('clf', LogisticRegression(solver='liblinear'))
    ])

    grid = GridSearchCV(pipeline, parametros, cv=3)
    grid.fit(X_train, y_train)

    y_pred = grid.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)

    print("Precisión:", acc)
    print("Matriz de confusión:")
    print(cm)

# Ejecutar con profiling
import cProfile
cProfile.run('ejecutar_modelo()')


c:\Users\Usuario\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Precisión: 0.0
Matriz de confusión:
[[0 1]
 [2 0]]
         705636 function calls (690258 primitive calls) in 1.219 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    1.223    1.223 949260922.py:18(ejecutar_modelo)
        3    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(all)
        1    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(amax)
        2    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(amin)
      182    0.001    0.000    0.006    0.000 <__array_function__ internals>:177(any)
        1    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(argsort)
       55    0.000    0.000    0.010    0.000 <__array_function__ internals>:177(average)
        4    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(bincount)
        2    0.000    0.000    0.001    0.000 <__array_function__ internals>:

# Ejercicio 10

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report, confusion_matrix

# Cargar los datos
df = pd.read_csv('train.csv')

# 1. Preprocesamiento
# Codificar la variable categórica objetivo (precio de la casa)
encoder = LabelEncoder()
df['price_category'] = pd.cut(df['SalePrice'], bins=[0, 100000, 200000, float('inf')], labels=['barata', 'media', 'cara'])
df['price_category'] = encoder.fit_transform(df['price_category'])

# 2. Eliminar filas con NaN o imputar valores faltantes
# Imputar valores faltantes con la media de cada columna
imputer = SimpleImputer(strategy='mean')
X = df.drop(['SalePrice', 'price_category', 'Id'], axis=1)  # Eliminar columnas no relevantes
y = df['price_category']

# Imputar valores faltantes en X
X_imputed = imputer.fit_transform(X.select_dtypes(include=['float64', 'int64']))

# 3. Escalado de características
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

# 4. Dividir en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# 5. Definir el modelo de regresión logística
model = LogisticRegression()

# 6. Realizar una búsqueda en cuadrícula para encontrar los mejores parámetros
param_grid = {
    'C': [0.01, 0.1, 1, 10],
    'solver': ['lbfgs', 'saga'],
    'max_iter': [100, 200, 300]
}

grid_search = GridSearchCV(model, param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Mejor modelo
best_model = grid_search.best_estimator_

# 7. Evaluar el modelo
y_pred = best_model.predict(X_test)

# 8. Imprimir resultados
print(f"Mejores parámetros: {grid_search.best_params_}")
print("\nMatriz de confusión:")
print(confusion_matrix(y_test, y_pred))
print("\nInforme de clasificación:")
print(classification_report(y_test, y_pred))


Mejores parámetros: {'C': 10, 'max_iter': 100, 'solver': 'saga'}

Matriz de confusión:
[[ 18   0   8]
 [  0  69  13]
 [  4  11 169]]

Informe de clasificación:
              precision    recall  f1-score   support

           0       0.82      0.69      0.75        26
           1       0.86      0.84      0.85        82
           2       0.89      0.92      0.90       184

    accuracy                           0.88       292
   macro avg       0.86      0.82      0.84       292
weighted avg       0.88      0.88      0.88       292



c:\Users\Usuario\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
